# Virally Suppressed Feature Engineering
**Description:** This script prepares the lab data and creates the several virally suppressed target based on 1000 cp/ml and undetectable 40cp/ml. Feature Engineering is also performed on the dataset.

**Author:** Marang Mutloatse

**Version:** 0.0.1

**Status:** Development

# Import Libraries

In [1]:
import os
# get parent user path
user_dev_path = os.path.dirname(os.getcwd())
# get parent path for package
package_path = os.path.dirname(user_dev_path)

In [2]:
import sys
sys.path.insert(0,package_path + '/src/')
import pandas as pd
import datetime as dt
import warnings
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import math
import re
import yaml
from hiv_support_package import visit_features, viral_load_targets
from sklearn.preprocessing import LabelEncoder
from datetime import timedelta
plt.style.use('fivethirtyeight')
warnings.simplefilter(action='ignore', category=FutureWarning)
%matplotlib inline

## Loading Functions

In [3]:
try: 
    with open (user_dev_path + '/' + 'config_LTFU.yaml', 'r') as file:
        config = yaml.safe_load(file)
except Exception as e:
    print(f'Error reading the config file, {e}')

In [4]:
input_root = config['rise_files']['raw_data_path']
input_file = config['rise_files']['raw_lab_file']

input_criteria_start = config['vl_inclusion_criteria']['start_taken_date']
input_criteria_end = config['vl_inclusion_criteria']['end_taken_date']

lab_input = input_root + input_file 
print(lab_input)
print(" ")
print(f"Input start date is: {input_criteria_start} \n")
print(f"Input end date is: {input_criteria_end} \n")

/data/rise_data/All LaboratoryData_Flat File.csv
 
Input start date is: 2015 

Input end date is: 2021 



In [5]:
df = pd.read_csv(lab_input,low_memory=False)

## Working Functions

In [6]:
def check_null_values(df):
    if df is not None:
        try:
            missing_value = df.isnull().sum()
            mis_val_percent = 100 * df.isnull().sum() / len(df)
            mis_val_table = pd.concat([missing_value, mis_val_percent], axis=1)
            mis_val_table_rename_cols = mis_val_table.rename(
                columns={0: "Missing Values", 1: "% of Total Values"}
            )
            mis_val_table_rename_cols = (
                mis_val_table_rename_cols[mis_val_table_rename_cols.iloc[:, 1] != 0]
                .sort_values("% of Total Values", ascending=False)
                .round(1)
            )
            print(
                "There are "
                + str(mis_val_table_rename_cols.shape[0])
                + " columns that have missing values"
            )
            return mis_val_table_rename_cols
        except Exception as e:
            print(f"Exception on checking missing values with error: {e}")

def print_num_rows_cols(df, df_name: str):
        if df is not None:
            return print(f"{df_name} data has {df.shape[1]} columns and {df.shape[0]} rows.") 
    
def one_hot_encode_col(df,dummies_col:str,prefix_entry:str):
    if df is not None:
        try:
            dummies = pd.get_dummies(df[dummies_col], prefix=prefix_entry,drop_first=False)
            df = df.join(dummies)
            return df
        except Exception as e:
            print(f"Exception on one hot encoding with error: {e}") 

# Initial Data Check

In [7]:
print_num_rows_cols(df,'Labs')

Labs data has 13 columns and 813244 rows.


In [8]:
check_null_values(df)

There are 6 columns that have missing values


Missing Values  % of Total Values
RESULTPC                740919               91.1
COMMENT                 632149               77.7
LABNO                   321830               39.6
resultab                 27706                3.4
DATE_COLLECTED            5919                0.7
DATE_REPORTED              839                0.1

In [9]:
print('Patients:',df['PATIENT_ID'].nunique())

Patients: 103401


In [10]:
df['Labtest Name'] = df['Labtest Name'].dropna()
df['Labtest Name'].describe()

count         813244
unique            37
top       Viral Load
freq          209459
Name: Labtest Name, dtype: object

In [11]:
# Filter for Viral Load Tests
df = df.loc[df['Labtest Name'].isin(['Viral Load'])]

## Data Cleaning

In [12]:
# Ensure target column is string format
df['resultab'] = df['resultab'].astype(str)

In [13]:
df.loc[(df['resultab'] == '<400') | (df['resultab'] == ' <400 ') ,'resultab'] = '400'
df.loc[(df['resultab'] == '<40') | (df['resultab'] == '<40') ,'resultab'] = '40'
df.loc[(df['resultab'] == '<400 cp/ml ') | (df['resultab'] == '<400cp/ml') | (df['resultab'] == '<400 C/ML'),'resultab'] = '400'

In [14]:
df.loc[df['resultab'].str.contains("not detec|not det|nt det|nt dec|nt detec|nd|undetected|tnd|T N D|TND|TDN|N.D.|ND|N/D|NOT DETECT"),'resultab']='40'

In [15]:
# coercing nan strings, to nan values
df.loc[(df['resultab'] == 'nan') | (df['resultab'] == 'na') ,'resultab'] = pd.np.nan

In [16]:
#  extracting viral load in integer and floats
df['resultab'] = df['resultab'].str.extract(r'(\d+\.\d+|\d+)', expand=False)

In [17]:
# cleaning up trailing whitespaces
df['resultab'] = df['resultab'].replace(r'^\s*$', pd.np.nan, regex=True)

In [18]:
check_null_values(df)

There are 5 columns that have missing values


Missing Values  % of Total Values
RESULTPC               198685               94.9
LABNO                   42524               20.3
COMMENT                 36542               17.4
resultab                  779                0.4
DATE_REPORTED             772                0.4

##  Deduplicating Data

In [19]:
df.duplicated().sum()

0

In [20]:
print(f"There are {df.loc[:, df.columns != 'LABORATORY_ID'].duplicated(subset=['DATE_COLLECTED','PATIENT_ID','DATE_REPORTED','resultab']).sum()} duplicate rows based on the DATE_COLLECTED,PATIENT_ID,DATE_REPORTED and resultab fields")

There are 6276 duplicate rows based on the DATE_COLLECTED,PATIENT_ID,DATE_REPORTED and resultab fields


In [21]:
print(f"Before deduplication the dataframe has {df.shape[0]} rows")
print(" ")
df.drop_duplicates(subset=['PATIENT_ID','DATE_REPORTED','DATE_COLLECTED','resultab'],inplace=True)
print(f"After deduplication the dataframe has {df.shape[0]} rows")

Before deduplication the dataframe has 209459 rows
 
After deduplication the dataframe has 203183 rows


In [22]:
# Choosing notnull(0)
df = df[df['resultab'].notnull()]
df['resultab'] = df['resultab'].astype(float)

In [23]:
print(f"After drop null values the dataframe has {df.shape[0]} rows")

After drop null values the dataframe has 202422 rows


## Data Preparation

In [24]:
# Clean up comment column
df['COMMENT'] = df['COMMENT'].str.replace(' ','Routine Monitoring')

comment_col = {
    "Routine Monitoring":"Routine Monitoring",
    "Target Monitoring":"Targeted Monitoring",
    "Targeted Monitoring":"Targeted Monitoring"
}

comment_col_label = {
    "Routine Monitoring":0,
    "Unspecified":1,
    "Targeted Monitoring":0,
}

df['COMMENT'] = df['COMMENT'].map(comment_col)
df['COMMENT'] = df['COMMENT'].fillna("Unspecified")
df['COMMENT'] = df['COMMENT'].map(comment_col_label)

In [25]:
# Convert to datetime
df['DATE_COLLECTED'] = pd.to_datetime(df['DATE_COLLECTED'],format="%d/%m/%Y",errors='coerce')
df['DATE_REPORTED'] = pd.to_datetime(df['DATE_REPORTED'],format="%d/%m/%Y",errors='coerce')

In [26]:
print(df[df['Labtest Name'] =='Viral Load']['resultab'].dropna().value_counts(
    bins=[0,1, 40, 200,500,1000,1600,2000,3000,1000000000]).sort_index())
n = len(df)
print(df.shape)
df = df[~((df['Labtest Name']=='Viral Load') & (df['resultab'] > 10000000))]
n = n - len(df)
print(df.shape, n, 'Tests removed removed')

(-0.001, 1.0]              12032
(1.0, 40.0]               113800
(40.0, 200.0]              28348
(200.0, 500.0]             10760
(500.0, 1000.0]             5393
(1000.0, 1600.0]            2736
(1600.0, 2000.0]            1217
(2000.0, 3000.0]            2056
(3000.0, 1000000000.0]     26080
Name: resultab, dtype: int64
(202422, 13)
(202413, 13) 9 Tests removed removed


In [27]:
df = df.sort_values(by =['PATIENT_ID','DATE_REPORTED']) 

In [28]:
df.shape

(202413, 13)

In [29]:
df = df[df['DATE_REPORTED'].notnull()]
df = df[df['DATE_COLLECTED'].notnull()]

In [30]:
df.shape

(201790, 13)

### Date Time conversion and Feature Creation

In [31]:
# changed from collected to reported or vice-versa
# df['EF_TAKEN_YEAR'] = df['DATE_REPORTED'].dt.year
# df['EF_TAKEN_MONTH'] = df['DATE_REPORTED'].dt.month
df = visit_features.generate_date_features(df,'DATE_REPORTED')
#df = visit_features.generate_date_features(df,'DATE_COLLECTED')

Generating features for DATE_REPORTED, starting with  rows:201,790; cols: 13
Generating features for DATE_REPORTED,ending with  rows:201,790; cols: 25


In [32]:
rename_dict = {"EF_ISSTARTOFMONTH_DATE_REPORTED":"EF_IS_START_OF_MONTH_DATE_REPORTED",
               "EF_ISENDOFMONTH_DATE_REPORTED":"EF_IS_END_OF_MONTH_DATE_REPORTED",
               "EF_DAYOFMONTH_DATE_REPORTED":"EF_DAY_OF_MONTH_DATE_REPORTED",
               "EF_DAYOFWEEK_DATE_REPORTED":"EF_DAY_OF_WEEK_DATE_REPORTED",
              "EF_ISWEEKEND_DATE_REPORTED":"EF_IS_WEEKEND_DATE_REPORTED",
              "EF_WEEKOFMONTH_DATE_REPORTED":"EF_WEEK_OF_MONTH_DATE_REPORTED",
              "EF_ISSTARTOFQUARTER_DATE_REPORTED":"EF_IS_START_OF_QUARTER_DATE_REPORTED",
              "EF_ISENDOFQUARTER_DATE_REPORTED":"EF_IS_END_OF_QUARTER_DATE_REPORTED"}
df = df.rename(columns=rename_dict)

### Inclusion Criteria

In [33]:
# Inclusion years
df = df[df['EF_YEAR_DATE_REPORTED']>int(input_criteria_start)]
df = df[df['EF_YEAR_DATE_REPORTED']<=int(input_criteria_end)]

# Target Creation

In [34]:
# Generate virally undetectable target
df = viral_load_targets.virally_undetectable_target(df, 'resultab', 40, 'Labtest Name', 'Viral Load')
df = viral_load_targets.vl_categorization_rise_target(df, 'resultab', 40, 1000)

# Generate virally suppressed target
df = viral_load_targets.virally_suppressed_target(df, 'resultab', 1000, 'Labtest Name', 'Viral Load')
df = viral_load_targets.virally_suppressed_target(df, 'resultab', 400, 'Labtest Name', 'Viral Load')

# Generate virally unsuppressed target
df = viral_load_targets.virally_unsuppressed_target(df, 'resultab', 1000, 'Labtest Name', 'Viral Load')
df = viral_load_targets.virally_unsuppressed_target(df, 'resultab', 400, 'Labtest Name', 'Viral Load')

In [35]:
# Calculate the RISE Target bands (below =< 40cp/ml & 40 - 1000 cp/ml & > 1000 cp/ml)
target_definitions = [(df['resultab'] <= 40),(df['resultab'] > 40) & (df['resultab'] < 1000),(df['resultab'] >=1000)]
target_values = ['Undetectable','Suppressed','Unsuppressed']
df['OUTCOME_RISE_VL_CATEGORIZATION'] = np.select(target_definitions,target_values)

# Engineered Features

In [36]:
df['EF_LAB_TYPE_COUNT'] = df.groupby(['PATIENT_ID','Labtest Name'])['resultab'].cumcount()+1
df['EF_VL_COUNT'] = 0
df.loc[df['Labtest Name']=='Viral Load','EF_VL_COUNT'] = df['EF_LAB_TYPE_COUNT']

In [37]:
# question: clarity on definition
df['EF_LAST_LAB_VALUE'] = df.groupby(['PATIENT_ID','Labtest Name'])['resultab'].shift(1)
df['EF_LAST_VL_VALUE'] = -1
df.loc[df['Labtest Name']=='Viral Load','EF_LAST_VL_VALUE'] = df['EF_LAST_LAB_VALUE']

In [38]:
# remove for preanalysis of viral loads,'EF_LAST_CD4A_VALUE and 'EF_LAST_CD4A_VALUE'
df[['EF_LAST_LAB_VALUE','EF_LAST_VL_VALUE']] = df[['EF_LAST_LAB_VALUE','EF_LAST_VL_VALUE']].fillna(-1)

In [39]:
df['EF_VL_VAL'] = df[df['Labtest Name']=='Viral Load']['resultab']
df['EF_VL_VAL'] = df['EF_VL_VAL'].fillna(-2)  

#ok, so this fillna is key as the next line gets confused if there are nans in the mix
df['EF_VL_CUMMAX'] = df.groupby(['PATIENT_ID'])['EF_VL_VAL'].cummax()
df['EF_VL_CUMMAX'] = df.groupby(['PATIENT_ID'])['EF_VL_CUMMAX'].shift(1)
df['EF_VL_CUMMAX'] = df['EF_VL_CUMMAX'].fillna(-2)

In [40]:
df.loc[df['EF_VL_VAL'] < 0,'EF_VL_VAL'] = 9999999
df['EF_VL_CUMMIN'] = df.groupby(['PATIENT_ID'])['EF_VL_VAL'].cummin()
df['EF_VL_CUMMIN'] = df.groupby(['PATIENT_ID'])['EF_VL_CUMMIN'].shift(1)
df['EF_VL_CUMMIN'] = df['EF_VL_CUMMIN'].fillna(-2)
df.loc[df['EF_VL_CUMMIN']==9999999,'EF_VL_CUMMIN'] = -2

In [41]:
df[df['Labtest Name']=='Viral Load'].EF_LAST_VL_VALUE.value_counts(bins=[-100,0,400,1000,10000,100000000],normalize=True).sort_index()

(-100.001, 0.0]           0.430692
(0.0, 400.0]              0.421659
(400.0, 1000.0]           0.023745
(1000.0, 10000.0]         0.044660
(10000.0, 100000000.0]    0.079244
Name: EF_LAST_VL_VALUE, dtype: float64

In [42]:
# added but might not be necessary
# df['EF_NEXT_VL_DATE_EST'] = df['DATE_REPORTED'] + np.timedelta64(1, 'Y')
# df['EF_NEXT_VL_DATE_COLLECTED'] = df['DATE_COLLECTED'] +  np.timedelta64(1, 'Y')

# # Taken Date
# df['EF_NEXT_ACTUAL_VL_DATE'] = df.groupby('PATIENT_ID')['DATE_REPORTED'].shift(-1)
# df['EF_DAYS_BETWEEN_NEXT_VL_AND_ACTUAL_VL'] = df['EF_NEXT_ACTUAL_VL_DATE'] - df['EF_NEXT_VL_DATE_EST']

# # Collected Dates
# df['EF_NEXT_ACTUAL_VL_DATE_COLLECTED'] = df.groupby('PATIENT_ID')['DATE_COLLECTED'].shift(-1)
# df['EF_DAYS_BETWEEN_NEXT_VL_AND_ACTUAL_VL_COLLECTED'] = df['EF_NEXT_ACTUAL_VL_DATE_COLLECTED'] - df['EF_NEXT_VL_DATE_COLLECTED']

## Exporting Columns

In [43]:
drop_cols = [
    "RESULTPC",
    "LABNO",
    "L.G.A",
    "OUTCOME_RISE_VL_CATEGORIZATION",
    "State",
    "Labtest Name",
    "Facility Name",
    "EF_LAB_TYPE_COUNT",
    "EF_LAST_LAB_VALUE",
]
df.drop(columns=drop_cols,inplace=True)

In [44]:
check_null_values(df)

There are 0 columns that have missing values


Empty DataFrame
Columns: [Missing Values, % of Total Values]
Index: []

In [45]:
# Replace spaces in columns names
df.columns = df.columns.str.replace(' ','_')

# Standardizing columns
df.columns = map(lambda x: str(x).upper(), df.columns) 

In [47]:
df = df.add_prefix('L_')

In [49]:
df.to_csv(input_root+'feature_labs.csv',index=False,header=True)